# Schema-based information retrieval from judgements


In [ ]:
# | eval: false
%load_ext autoreload
%autoreload 2

In [ ]:
# | eval: false
from loguru import logger

import mlflow

from juddges.prompts.information_extraction import (
    prepare_information_extraction_chain,
    EXAMPLE_SCHEMA,
)
from juddges.settings import prepare_langchain_cache, prepare_mlflow
from juddges.data.pl_court_api import PolishCourtAPI

In [ ]:
# | eval: false
from dotenv import load_dotenv

load_dotenv("../../.env")

True

In [ ]:
# |eval: false
prepare_langchain_cache()
prepare_mlflow()

2024/04/10 08:12:59 INFO mlflow.tracking.fluent: Experiment with name 'Juddges-Information-Extraction' does not exist. Creating a new experiment.


In [ ]:
# | eval: false
logger.add(level="INFO", sink="../../logs/ir_judgments.log")

1

In [ ]:
LLM_NAME = "gpt-4-0125-preview"
PROMPT_VERSION = "initial-ir"
LANGUAGE = "Polish"

In [ ]:
# | eval: false
mlflow.start_run(run_name=f"{PROMPT_VERSION}-{LLM_NAME}")

<ActiveRun: >

In [ ]:
# | eval: false
api = PolishCourtAPI()

In [ ]:
# | eval: false
judgement = api.get_content(id="155000000001006_II_AKa_000334_2019_Uz_2020-02-06_001")

In [ ]:
# | eval: false
judgement

{'content': '<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<xPart xPublisherFullName="Anna Turek" xPublisher="anna.turek" xEditorFullName="Wiktoria Dąbrowicz" xEditor="wiktoria.dabrowicz" xVersion="1.0" xLang="PL" xFromPg="1" xToPage="2" xFlag="published" xVolType="15/500000/0001006/AKa" xYear="2019" xVolNmbr="000334" xDocType="Uz" xml:space="preserve">\n  <xName>Wyrok</xName>\n  <xBlock>\n    <xText xALIGNx="left">\n      <xBx>Sygn. akt II AKa 334/19</xBx></xText>\n    <xText/>\n    <xUnit xBold="true" xIsTitle="true" xType="part">\n      <xName>WYROK</xName>\n      <xTitle>W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ</xTitle>\n      <xText xALIGNx="right">Dnia 6 lutego 2020 r.</xText>\n      <xText><xBx>Sąd Apelacyjny we Wrocławiu II Wydział Karny w składzie:</xBx></xText>\n      <xText>Przewodniczący: sędzia SA Cezariusz Baćkowski</xText>\n      <xText>Sędziowie: SA Piotr Kaczmarek (spr.)</xText>\n      <xText>SA Robert Zdych</xText>\n      <xText>Protokolant: Wiktoria Dąbrowicz</xText>\n     

In [ ]:
# | eval: false
print(EXAMPLE_SCHEMA)


verdict_date: date as ISO 8601
verdict: string, text representing verdict of the judgement
verdict_summary: string, short summary of the verdict
verdict_id: string
court: string
parties: string
appeal_against: string
first_trial: boolean
drug_offence: boolean
child_offence: boolean
offence_seriousness: boolean
verdict_tags: List[string]



In [ ]:
# | eval: false
chain = prepare_information_extraction_chain()

In [ ]:
# | eval: false
logged_model = mlflow.langchain.log_model(chain, "langchain_model")

2024/04/10 08:13:25 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmp99h0ktw2/model, flavor: langchain). Fall back to return ['langchain==0.1.13', 'pydantic==2.6.4', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 


In [ ]:
# | eval: false
mlflow.log_param("prompt_version", PROMPT_VERSION)
mlflow.log_param("llm_name", LLM_NAME)
mlflow.log_param("language", LANGUAGE)
mlflow.log_param("schema", EXAMPLE_SCHEMA)

'\nverdict_date: date as ISO 8601\nverdict: string, text representing verdict of the judgement\nverdict_summary: string, short summary of the verdict\nverdict_id: string\ncourt: string\nparties: string\nappeal_against: string\nfirst_trial: boolean\ndrug_offence: boolean\nchild_offence: boolean\noffence_seriousness: boolean\nverdict_tags: List[string]\n'

In [ ]:
# | eval: false
retrieved_informations = chain.invoke(
    {"LANGUAGE": LANGUAGE, "TEXT": judgement, "SCHEMA": EXAMPLE_SCHEMA}
)

In [ ]:
# | eval: false
retrieved_informations

{'verdict_date': '2020-02-06',
 'verdict': 'zmienia zaskarżony wyrok',
 'verdict_summary': 'Sąd Apelacyjny we Wrocławiu zmienia wyrok Sądu Okręgowego w Świdnicy, uniewinniając częściowo oskarżonych i utrzymując w mocy pozostałą część wyroku.',
 'verdict_id': 'II AKa 334/19',
 'court': 'Sąd Apelacyjny we Wrocławiu II Wydział Karny',
 'parties': 'A. D., R. M., E. L., R. P., L. L.',
 'appeal_against': 'wyrok Sądu Okręgowego w Świdnicy z dnia 5 kwietnia 2019 r., sygn. akt III K 52/17',
 'first_trial': False,
 'drug_offence': True,
 'child_offence': False,
 'offence_seriousness': True,
 'verdict_tags': ['uniewinnienie',
  'zmiana wyroku',
  'przestępstwo narkotykowe']}

In [ ]:
# | eval: false
from juddges.data.models import get_mongo_collection

judgements_collection = get_mongo_collection("judgements")

In [ ]:
# | eval: false
judgements = list(judgements_collection.find(filter={}).limit(100))

In [ ]:
# | eval: false
len(judgements)

100

Estimate price of prompt with judgements text for 100 sample (is USD)


In [ ]:
# | eval: false
from juddges.settings import num_tokens_from_string, LLM_TO_PRICE_INPUT

sum(
    [
        LLM_TO_PRICE_INPUT[LLM_NAME] * num_tokens_from_string(judgement["text"])
        for judgement in judgements
    ]
)

3.1003800000000017

In [ ]:
# | eval: false
print(judgements[0]["text"])

Sygn. akt. XVII Amr 14/94

WYROK
W IMIENIU RZECZYPOSPOLITEJ POLSKIEJ
Dnia 20 lipca 1994 r.
Sąd Wojewódzki w Warszawie - Sąd Antymonopolowy w składzie następującym:
Przewodniczący SSW Stanisław Gronowski (spr.)
Sędziowie: SW Jacek Sikorski i SR Bogdan Gierzyński
Protokólant apl. rade. A. Grześkiewicz
po rozpoznaniu w dniu 6 lipca 1994 r. w Warszawie na rozpra­wie
spraw z odwołań:A. M.,B. U.
Sklep (...)weW.,Przedsiębiorstwa Handlowego (...) s.c.weW.,W. K.iA. B.(...)weW.
od decyzji Urzędu Antymonopo1owego Delegatury weW.z dnia 29 października 1993 r., sygn. akt(...)przeciwko Urzędowi Antymonopolowemu zainteresowany: GminaW.
o przeciwdziałanie praktykom monopolistycznym

1
odwołania oddala

2
zasądza na rzecz Urzędu Antymonopolowego od:

a
A. M.kwotę 5.000.000 (pięć milionów) złotych

b
B. U.Sklep (...)weW.kwotę 5.000.000 (pięć milionów) złotych

c
Przedsiębiorstwa Handlowego (...) s.c.weW.kwotę 5.000.000 (pięć milionów) złotych

(...)-

d

W. K.iA. B.(...)s.c. weW.kwotę 5.000.000 (pięć mi

In [ ]:
# | eval: false
# save sample judgments
import pandas as pd
from juddges.settings import SAMPLE_DATA_PATH

In [ ]:
# | eval: false
df = pd.DataFrame(judgements)
df.to_csv(SAMPLE_DATA_PATH / "judgements-100-sample.csv")

In [ ]:
# | eval: false
df.head(2)

,_id,signature,date,publicationDate,lastUpdate,courtId,departmentId,type,excerpt,content,...,legalBases,publisher,recorder,references,reviser,themePhrases,num_pages,text,vol_number,vol_type
0,154505000005127_XVII_AmA_000014_1994_Uz_1994-0...,XVII AmA 14/94,1994-07-20 02:00:00.0 CEST,2018-06-13 22:10:05.0 CEST,2018-06-13 16:22:22.0 CEST,15450500,5127,"SENTENCE, REASON",Sygn. akt. XVII Amr 14/94 WYROK W IMIENIU RZEC...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,{'legalBasis': ['art. 6 ust. 1 pkt 1 ustawy oc...,Beata Gonera,None,{'reference': ['Ustawa z dnia 17 listopada 196...,Beata Gonera,{'themePhrase': ['Praktyki ograniczające konku...,7,Sygn. akt. XVII Amr 14/94\n\nWYROK\nW IMIENIU ...,14,15/450500/0005127/AmA
1,151500000001006_II_AKa_000031_1998_Uz_2013-02-...,II AKa 31/98,1998-03-26 01:00:00.0 CET,2013-02-08 02:01:01.0 CET,2013-12-04 17:23:33.0 CET,15150000,1006,"SENTENCE, REASON",Sygn. akt : II AKa 31/98 WYROK W IMIENIU RZECZ...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,{'legalBasis': 'art. 1 ustawy o ochronie obrot...,Renata Kopiec,Barbara Gawor,{'reference': ['Ustawa z dnia 12 października ...,Magdalena Gierszner,{'themePhrase': 'Przestępstwo Przeciwko Obroto...,13,Sygn. akt : II AKa 31/98\n\nWYROK\nW IMIENIU R...,31,15/150000/0001006/AKa


In [ ]:
# | eval: false
judgements_retrieved_informations = chain.batch(
    [
        {"LANGUAGE": LANGUAGE, "TEXT": judgement["text"], "SCHEMA": EXAMPLE_SCHEMA}
        for judgement in judgements
    ]
)

In [ ]:
# | eval: false

# add list of retrieved informations to each judgement to the data frame df but treat each key in dict as a new column
from tqdm import tqdm

for key in tqdm(judgements_retrieved_informations[0].keys(), desc="Adding columns to DataFrame"):
    df[key] = [judgement[key] for judgement in judgements_retrieved_informations]

Adding columns to DataFrame:   0%|          | 0/12 [00:00<?, ?it/s]

Adding columns to DataFrame: 100%|██████████| 12/12 [00:00<00:00, 3055.03it/s]


In [ ]:
# | eval: false
df.head(2)

,_id,signature,date,publicationDate,lastUpdate,courtId,departmentId,type,excerpt,content,...,verdict_summary,verdict_id,court,parties,appeal_against,first_trial,drug_offence,child_offence,offence_seriousness,verdict_tags
0,154505000005127_XVII_AmA_000014_1994_Uz_1994-0...,XVII AmA 14/94,1994-07-20 02:00:00.0 CEST,2018-06-13 22:10:05.0 CEST,2018-06-13 16:22:22.0 CEST,15450500,5127,"SENTENCE, REASON",Sygn. akt. XVII Amr 14/94 WYROK W IMIENIU RZEC...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,Sąd Antymonopolowy oddalił odwołania wniesione...,XVII Amr 14/94,Sąd Wojewódzki w Warszawie - Sąd Antymonopolowy,"A. M., B. U., Sklep (...) we W., Przedsiębiors...",decyzji Urzędu Antymonopolowego Delegatury we ...,False,False,False,False,"[praktyki monopolistyczne, umowy najmu, przeta..."
1,151500000001006_II_AKa_000031_1998_Uz_2013-02-...,II AKa 31/98,1998-03-26 01:00:00.0 CET,2013-02-08 02:01:01.0 CET,2013-12-04 17:23:33.0 CET,15150000,1006,"SENTENCE, REASON",Sygn. akt : II AKa 31/98 WYROK W IMIENIU RZECZ...,<?xml version='1.0' encoding='UTF-8'?>\n<xPart...,...,Sąd Apelacyjny w Katowicach uniewinnił oskarżo...,II AKa 31/98,Sąd Apelacyjny w Katowicach,"J. K., W. P., B. K.",wyrok Sądu Wojewódzkiego w Bielsku-Białej z dn...,False,False,False,True,"[uniewinnienie, nadużycie zaufania, szkoda maj..."


In [ ]:
# | eval: false
df.to_csv(SAMPLE_DATA_PATH / "judgements-100-sample-with-retrieved-informations.csv")